In [ ]:
import sys, os, glob, tqdm
from skimage.io import imread, imsave
import numpy as np
import napari

sys.path.append(os.path.join('C:',os.sep,'Users','Costanzo','Desktop','pymif'))
import pymif.image_preprocessing as pmpre
import pymif.cell_detection as pmcell


# Load image

In [ ]:
file_name = os.path.join('..','2022-07-19_154438_22x','processed_DAPI',
                         '20220719-162746_Task_4_Description_for_C',
                         'fused--C00--T00001.tif'
#                         'uni_tp-0_ch-1_st-0_obj-right_cam-right_etc.lux.h5'
                        )

In [ ]:
# img = pymif.open_h5(file_name)
img = imread(file_name)
print(img.shape)

In [ ]:
pixel_size = [2., 2*0.2925, 2*0.2925]

#desired isotropic resolution in um/pxl
iso_res = 1.

# size of individual tiles (chunks), better if factor
# img = img[1:] # to make shape even (390,2048,2048)
chunk_size = (128,512,512)

# overlap between neighbor tiles (to avoid edge effects)
overlap_width = (8,32,32)

In [ ]:
img_iso = pmpre.isores_dask(img, pixel_size, iso_res)
# print(img_iso.shape)

In [ ]:
viewer = napari.Viewer()
viewer.add_image(img_iso, scale=(1,1,1), name='somitoid')
napari.run()

In [ ]:
# import numpy as np
# imsave('iso_res.tif', img_iso.astype(np.uint16))

# Find cells

In [ ]:
cell_diameter = [5, 5, 5]
img_tophat = pmpre.tophat_cell_dask(img_iso, 2*np.array(cell_diameter))
img_dog = pmpre.dog_cell(img_tophat, cell_diameter)

In [ ]:
viewer = napari.Viewer()
viewer.add_image(img_iso, scale=(1,1,1), name='somitoid')
viewer.add_image(img_dog, scale=(1,1,1), name='somitoid_dog')
# viewer.add_points(df[['z','y','x']].to_numpy(), name='cells', opacity=0.3, scale=(1,1,1),face_color='red')
napari.run()

In [ ]:
import numpy as np
df = pmcell.cell_local_max(img_dog, np.array(cell_diameter)/2., thr_DoG = 10)

In [ ]:
viewer = napari.Viewer()
viewer.add_image(img_iso, scale=(1,1,1), name='somitoid')
viewer.add_image(img_dog, scale=(1,1,1), name='somitoid_dog')
viewer.add_points(df[['z','y','x']].to_numpy(), name='cells', opacity=1., scale=(1,1,1),face_color='red')
napari.run()